In [4]:
from tensorflow.keras.models import load_model
import tensorflow as tf
import numpy as np
import librosa
import librosa.display

In [172]:
def preprocessing(audio_file, mode):
    # we want to resample audio to 16 kHz
    sr_new = 16000 # 16kHz sample rate
    x, sr = librosa.load(audio_file, sr=sr_new)
    print(x.shape)
    print(x)
    # for i in range(100):
    #   print(f"{x[i]:.8f}")
    # padding sound 
    # because duration of sound is dominantly 20 s and all of sample rate is 22050
    # we want to pad or truncated sound which is below or above 20 s respectively
    max_len = 5 * sr_new  # length of sound array = time x sample rate
    print(x.shape)
    if x.shape[0] < max_len:
      # padding with zero
      pad_width = max_len - x.shape[0]
      x = np.pad(x, (0, pad_width))
    elif x.shape[0] > max_len:
      # truncated
      x = x[:max_len]
    print(x.shape)
    # print(x==normalized_data)
    if mode == 'mfcc':
      feature = librosa.feature.mfcc(y=x, sr=sr_new)
    print(len(feature))

    return feature

In [29]:
# Load the saved Keras model
model = load_model("code/my_model.h5")

# # Convert to TensorFlow Lite format
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()

# # Save the model as a .tflite file
# with open("model.tflite", "wb") as f:
#     f.write(tflite_model)

In [173]:
data = preprocessing("data\\orig\\1\\New Recording 13 [breath only].wav", mode = 'mfcc')
print(data.shape)
# data = data.reshape((-1, 20, 157, 1))

(180885,)
[ 8.9523628e-06 -6.6128032e-06 -1.5109366e-05 ...  0.0000000e+00
 -3.6379788e-11 -2.9103830e-11]
(180885,)
(80000,)
20
(20, 157)


In [114]:
from scipy.io import wavfile
import wave
sr, pcm_data = wavfile.read("data\\orig\\1\\New Recording 13 [breath only].wav")

C:\Users\home\AppData\Local\Temp\ipykernel_1288\1600509492.py:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, pcm_data = wavfile.read("data\\orig\\1\\New Recording 13 [breath only].wav")


In [179]:
sr, pcm_data_from_wavfile = wavfile.read("data\\orig\\1\\New Recording 13 [breath only].wav")
print(f"PCM Data Length from wavfile.read(): {len(pcm_data_from_wavfile)}")
print(f"Shape from wavfile.read(): {pcm_data_from_wavfile.shape}")

# Manually reading the raw bytes from the WAV file using wave.open()
with wave.open("data\\orig\\1\\New Recording 13 [breath only].wav", "rb") as wav_file:
    # Get the sample rate, number of channels, and sample width (bit depth)
    sr = wav_file.getframerate()    # Sample rate (Hz)
    num_channels = wav_file.getnchannels()  # Number of channels (1 for mono, 2 for stereo)
    sampwidth = wav_file.getsampwidth()  # Bytes per sample (e.g., 2 for 16-bit, 4 for 32-bit)
    num_frames = wav_file.getnframes()  # Number of frames in the audio
    # Read the PCM data (raw bytes) from the WAV file
    # print(wav_file._sampwidth != 1)
    raw_pcm_data = wav_file.readframes(num_frames)
# raw_pcm_data=raw_pcm_data[:960000]
print(num_frames)
print(len(raw_pcm_data))
# Convert raw bytes to a numpy array based on the sample width
if sampwidth == 2:  # 16-bit PCM (2 bytes per sample)
    pcm_data_from_raw = np.frombuffer(raw_pcm_data, dtype=np.int16)
print(len(pcm_data_from_raw))
# Reshape the PCM data if stereo (2 channels)
if num_channels == 2:
    pcm_data_from_raw = pcm_data_from_raw.reshape(-1, 2)  # Shape it to (num_samples, 2)

print(f"PCM Data Length from raw bytes: {len(pcm_data_from_raw)}")
print(f"Shape from raw bytes: {pcm_data_from_raw.shape}")
sr_orig = 48000  # Original sample rate (e.g., 44.1kHz)
sr_new = 16000   # Target sample rate (e.g., 16kHz)

resample_ratio = sr_new / sr_orig
print(len(pcm_data_from_wavfile))
print(len(pcm_data_from_raw))
# Resample the data
new_length = int(len(pcm_data_from_raw) * resample_ratio)
resampled_data = scipy.signal.resample(pcm_data_from_raw, new_length)
normalized_data = resampled_data.astype(np.float32) / 32768.0
# print(new_length)
normalized_data.shape
# Check if the length and shape match
# assert pcm_data_from_wavfile.shape == pcm_data_from_raw.shape, "Shapes do not match!"

C:\Users\home\AppData\Local\Temp\ipykernel_1288\3987412722.py:1: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, pcm_data_from_wavfile = wavfile.read("data\\orig\\1\\New Recording 13 [breath only].wav")


PCM Data Length from wavfile.read(): 542655
Shape from wavfile.read(): (542655, 2)
542655
2170620
1085310
PCM Data Length from raw bytes: 542655
Shape from raw bytes: (542655, 2)
542655
542655


(180885, 2)

In [166]:
80000/resample_ratio

240000.0

In [135]:
import struct

def read_wav_bytes(file_path):
    with open(file_path, "rb") as file:
        # Read the first 44 bytes of the WAV file header
        header = file.read(44)
        
        # Unpack the header information
        (chunk_id, chunk_size, format, subchunk1_id, subchunk1_size, audio_format, 
         num_channels, sample_rate, byte_rate, block_align, bits_per_sample, 
         subchunk2_id, subchunk2_size) = struct.unpack('<4sI4s4sIHHIIHH4sI', header)
        
        # Validate the header
        assert chunk_id == b'RIFF'
        assert format == b'WAVE'
        assert subchunk1_id == b'fmt '
        assert subchunk2_id == b'data'
        
        # Read the PCM data starting at the offset of the data subchunk
        pcm_data = file.read()
        
        return pcm_data

pcm_bytes = read_wav_bytes("data\\orig\\1\\New Recording 13 [breath only].wav")
print(len(pcm_bytes))  # Print the length of the PCM data


2170726


In [144]:
import struct

# Read the raw bytes from the WAV file
with open("data\\orig\\1\\New Recording 13 [breath only].wav", "rb") as file:
    raw_bytes = file.read()

# Extract metadata from the WAV header
HEADER_SIZE = 44  # Standard WAV header size

# Extract relevant header information
riff = raw_bytes[0:4]  # Should be 'RIFF'
file_size = struct.unpack('<I', raw_bytes[4:8])[0]  # Total file size
wave1 = raw_bytes[8:12]  # 'WAVE'
fmt = raw_bytes[12:16]  # 'fmt '
fmt_chunk_size = struct.unpack('<I', raw_bytes[16:20])[0]
audio_format = struct.unpack('<H', raw_bytes[20:22])[0]  # PCM is 1
num_channels = struct.unpack('<H', raw_bytes[22:24])[0]
sr = struct.unpack('<I', raw_bytes[24:28])[0]  # Sample rate (Hz)
byte_rate = struct.unpack('<I', raw_bytes[28:32])[0]
block_align = struct.unpack('<H', raw_bytes[32:34])[0]
sampwidth = struct.unpack('<H', raw_bytes[34:36])[0]  # Bits per sample (16-bit = 2 bytes)

# PCM data starts after the header
pcm_data = raw_bytes[HEADER_SIZE:]
for i in range(12):
    if(pcm_data[i]!=raw_pcm_data[i+1]):
        print("here")

# Calculate framesize (bytes per frame)
framesize = sampwidth * num_channels

# Calculate the total number of frames
total_frames = len(pcm_data) // framesize

# Output the results
print(fmt_chunk_size)
print(f"Total PCM Data Length: {len(pcm_data)} bytes")
print(f"Framesize (bytes per frame): {framesize}")
print(f"Total Frames: {total_frames}")

def readframes(nframes):
    start_index = 0
    # Read raw PCM data for nframes
    end_index = start_index + nframes * framesize
    data = pcm_data[start_index:end_index]
    return data

# Example: Read the first 1000 frames
nframes = total_frames
raw_data = readframes(nframes)

# Convert the raw byte data to a numpy array based on sample width (bit depth)
if sampwidth == 16:  # 16-bit PCM (2 bytes per sample)
    pcm_data_from_raw = np.frombuffer(raw_data, dtype=np.int16)
elif sampwidth == 4:  # 32-bit PCM (4 bytes per sample)
    pcm_data_from_raw = np.frombuffer(raw_data, dtype=np.int32)
else:
    raise ValueError(f"Unsupported sample width: {sampwidth}")

# Reshape the PCM data if stereo (2 channels)
if num_channels == 2:
    pcm_data_from_raw = pcm_data_from_raw.reshape(-1, 2)  # Shape it to (num_samples, 2)

# Output the results
print(f"PCM Data Length from raw bytes: {len(pcm_data_from_raw)}")
print(f"Shape from raw bytes: {pcm_data_from_raw.shape}")


here
here
here
here
16
Total PCM Data Length: 2170726 bytes
Framesize (bytes per frame): 32
Total Frames: 67835
PCM Data Length from raw bytes: 542680
Shape from raw bytes: (542680, 2)


In [94]:
2107026 - 1085310*2
1085363 -542655*2

53

In [ ]:


import scipy.signal

# Define original and target sample rates
sr_orig = 48000  # Original sample rate (e.g., 44.1kHz)
sr_new = 16000   # Target sample rate (e.g., 16kHz)

# Compute resampling ratio
resample_ratio = sr_new / sr_orig

# Resample the data
new_length = int(len(pcm_data_from_wavfile) * resample_ratio)
resampled_data = scipy.signal.resample(pcm_data_from_wavfile, new_length)

In [128]:
normalized_data = resampled_data.astype(np.float32) / 32768.0
len(normalized_data)

180885

In [ ]:
2170726
2170726/180885
180885*12

2170620

In [13]:
model.predict(data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step


array([[0.01240331, 0.9875967 ]], dtype=float32)